In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

from sklearn.model_selection import cross_validate, cross_val_score, cross_val_predict, learning_curve,\
train_test_split, GridSearchCV, RandomizedSearchCV, train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, OneHotEncoder, FunctionTransformer, LabelEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.inspection import permutation_importance
from sklearn.metrics import plot_confusion_matrix, classification_report, precision_recall_curve
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDRegressor, SGDClassifier, Ridge, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import SVC
from sklearn.compose import make_column_selector

## pipeline stuff

from sklearn.pipeline import Pipeline, make_pipeline, make_union
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn import set_config; set_config(display='diagram')



import warnings 
warnings.filterwarnings('ignore')

In [3]:
from tqdm.auto import tqdm
import itertools

In [4]:
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split

In [5]:
cat_vars = ['B_30', 
            'B_38', 
            'D_114', 
            'D_116', 
            'D_117', 
            'D_120', 
            'D_126', 
            'D_63', 
            'D_64', 
            'D_66', 
            'D_68']

In [6]:
df_pay = pd.read_csv("/home/slawa/code/code-rep0/projects/data/payer_data_41940.csv", index_col=[0])
df_def = pd.read_csv("/home/slawa/code/code-rep0/projects/data/defaulter_data_13364.csv", index_col=[0])

In [7]:
target_pay = df_pay.groupby('customer_ID').mean()
target_def = df_def.groupby('customer_ID').mean()

In [8]:
target_pay['target'] = int(0)
target_def['target'] = int(1) 

In [9]:
target = pd.concat([target_pay, target_def])

In [10]:
target = target.loc[:,'target']

In [11]:
target = target.reset_index()

In [12]:
df = pd.concat([df_pay, df_def])

In [13]:
df["c_PD_239"]=df["D_39"]/(df["P_2"]*(-1)+0.0001)
df["c_PB_29"]=df["P_2"]*(-1)/(df["B_9"]*(1)+0.0001)
df["c_PR_21"]=df["P_2"]*(-1)/(df["R_1"]+0.0001)

df["c_BBBB"]=(df["B_9"]+0.001)/(df["B_23"]+df["B_3"]+0.0001)
df["c_BBBB1"]=(df["B_33"]*(-1))+(df["B_18"]*(-1)+df["S_25"]*(1)+0.0001)
df["c_BBBB2"]=(df["B_19"]+df["B_20"]+df["B_4"]+0.0001)

df["c_RRR0"]=(df["R_3"]+0.001)/(df["R_2"]+df["R_4"]+0.0001)
df["c_RRR1"]=(df["D_62"]+0.001)/(df["D_112"]+df["R_27"]+0.0001)

df["c_PD_348"]=df["D_48"]/(df["P_3"]+0.0001)
df["c_PD_355"]=df["D_55"]/(df["P_3"]+0.0001)

df["c_PD_439"]=df["D_39"]/(df["P_4"]+0.0001)
df["c_PB_49"]=df["B_9"]/(df["P_4"]+0.0001)
df["c_PR_41"]=df["R_1"]/(df["P_4"]+0.0001)

In [14]:
df

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,c_BBBB,c_BBBB1,c_BBBB2,c_RRR0,c_RRR1,c_PD_348,c_PD_355,c_PD_439,c_PB_49,c_PR_41
0,e5a854a3211e83db521500e8b70360fe9670af1df90401...,2018-01-27,0.367170,0.005127,0.008564,1.006915,0.009815,0.226954,0.003249,0.000318,...,0.240582,-0.719884,0.026376,93.527964,0.041132,NaN,NaN,0.005312,0.002995,0.010169
1,e5a854a3211e83db521500e8b70360fe9670af1df90401...,2018-02-22,0.393798,0.008609,0.014054,1.000013,0.001827,0.222864,0.003984,0.006139,...,0.222950,-0.726073,0.069788,106.959299,0.037138,NaN,NaN,0.008921,0.008784,0.001893
2,e3ce8ab83161700aab304eb01f8b23385bea28dae59c8a...,2017-03-08,0.977155,0.002212,0.000951,0.819058,0.006936,0.313585,0.001432,0.014320,...,0.106694,-1.027956,0.087947,47.443315,0.378304,0.118312,0.141249,4.563409,5.537089,14.310557
3,e3ce8ab83161700aab304eb01f8b23385bea28dae59c8a...,2018-03-13,1.000395,0.186044,0.017830,0.813150,0.007263,0.123836,0.003332,0.000018,...,0.055488,-1.040972,0.052233,0.839227,0.748211,0.049795,0.261346,20.591684,0.021347,0.803855
4,e4531d93a40fe3892fd93c5d3991243acac7cfb90d2ce9...,2017-11-21,0.434304,0.009313,0.001116,0.819881,0.008974,0.177063,0.007866,0.005926,...,0.229565,-1.045192,0.088544,15.652176,0.091147,1.350078,0.689079,6.564925,1.647892,6.325904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13359,d7e2f69aa41106bb7413e9b3dcd28e1a0a074d2685d26a...,2017-11-18,0.342782,0.034313,0.026842,0.135342,0.008224,0.241999,0.006360,0.202902,...,0.271374,0.702368,0.547280,105.828241,0.058923,2.585518,2.234734,18.016712,57.799648,4.318215
13360,d7e2f69aa41106bb7413e9b3dcd28e1a0a074d2685d26a...,2017-12-19,0.339534,0.031273,0.023687,0.139861,0.001994,0.232908,0.009484,0.198701,...,0.223498,0.697835,0.555627,119.744388,0.019650,1.864197,1.670538,8.348961,24.184250,0.532409
13361,d7e2f69aa41106bb7413e9b3dcd28e1a0a074d2685d26a...,2018-01-19,0.226843,0.031205,0.021618,0.146228,0.008076,0.235040,0.003757,0.195985,...,0.198715,0.704181,0.567496,36.766743,0.023943,4.279048,3.889089,11.548969,29.617420,2.989068
13362,d7e2f69aa41106bb7413e9b3dcd28e1a0a074d2685d26a...,2018-02-16,0.230107,0.035024,0.021670,0.226883,0.509827,NaN,0.001564,0.175962,...,0.138995,0.767887,0.717766,85.181959,0.049961,3.732564,3.156621,13.324742,22.332714,193.963126


In [15]:
features = df.drop(['customer_ID', 'S_2'], axis = 1).columns.to_list() # get all feature names, except customer_ID and dates

In [16]:
num_features = [feature for feature in features if feature not in cat_vars]

In [17]:
train_num_agg = df.groupby("customer_ID")[num_features].agg(['mean', 'std', 'min', 'max', 'last']) # give summary statistics for each numerical feature
train_num_agg.columns = ['_'.join(x) for x in train_num_agg.columns] # join the column name tuples to a single name
train_num_agg.reset_index(inplace = True) # get the customer_ID in as a column again and reset index

In [18]:
train_num_agg

,customer_ID,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_mean,D_39_std,D_39_min,D_39_max,...,c_PB_49_mean,c_PB_49_std,c_PB_49_min,c_PB_49_max,c_PB_49_last,c_PR_41_mean,c_PR_41_std,c_PR_41_min,c_PR_41_max,c_PR_41_last
0,000919ba92d9a04c28e1e49f6cd855ca36e1df7c79cc05...,0.499402,0.015600,0.476616,0.522344,0.498361,0.470466,0.198861,0.004601,0.830081,...,225.297543,213.187017,64.916394,696.014237,125.073946,10.058885,22.090925,0.001433,67.413339,0.259988
1,00158cf08fcf7ec058529dd71b4cff04ce89314e79840b...,0.691427,0.024675,0.666900,0.733644,0.707521,0.026813,0.080476,0.000230,0.294493,...,1.827795,3.524579,0.001253,11.108380,3.967306,1.004138,1.743268,0.001401,4.546990,3.423076
2,0018753794d55ff4ab24aaa7f8e65d504fd1efbf04e369...,0.281522,0.090988,0.198344,0.498234,0.325086,0.095091,0.185570,0.001569,0.567713,...,316.086714,580.285457,0.613235,2192.639080,30.993720,24.989575,42.628358,0.006441,114.600555,0.949760
3,0044d8693a5c204d5e22297b11e566d9de1c1610899d20...,0.863033,0.035847,0.808965,0.915033,0.878732,0.005821,0.002665,0.001335,0.009564,...,0.895997,0.657292,0.160506,2.215508,2.215508,0.837358,0.695346,0.004019,2.406603,2.406603
4,00597bc3d552264d841bd1a52cfaf3ebe40755f96d85a5...,0.343063,0.047509,0.220225,0.378734,0.371411,0.164823,0.201824,0.009121,0.803373,...,298.391624,511.164682,59.923864,1947.667927,122.563743,127.859961,221.530941,27.114767,844.638848,54.379913
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4570,ffbdeb593c97bc39bf6228e7236796cf5dcd530ef73014...,0.800766,0.054486,0.665637,0.855915,0.823051,0.551746,0.366059,0.004717,1.215695,...,1.925914,3.953065,0.031917,14.654292,0.839985,2.738269,4.365143,0.241016,14.539556,0.907417
4571,ffbff903bdd4104397101a428a4c5c4daa4d28e26f8716...,0.985665,0.020549,0.947392,1.004385,1.004385,0.121927,0.116296,0.000431,0.383494,...,65.553615,180.477812,0.121604,665.090301,0.121604,5.038310,15.112799,0.094932,55.298213,0.100152
4572,ffd4ac1fe3746d586e00906a56ae8a26fdad358f5b4789...,0.583752,0.079215,0.441809,0.716586,0.537270,0.303983,0.264887,0.001018,0.886130,...,4.226561,3.724987,0.151578,9.621307,1.134810,1.466304,2.095533,0.244888,8.276437,0.731602
4573,ffd6a301b0f94dec47dda9fec844cdc640a04bb38e8707...,0.371422,0.028512,0.332961,0.422435,0.375672,0.057875,0.083183,0.005337,0.243254,...,5.333977,17.184645,0.341976,62.526558,0.341976,0.085933,0.196977,0.000415,0.538306,0.009001


In [19]:
def get_difference(data, num_features):
    df1 = []
    customer_ids = []
    for customer_id, df in tqdm(data.groupby(['customer_ID'])):
        # Get the differences
        diff_df1 = df[num_features].diff(1).iloc[[-1]].values.astype(np.float32)
        # Append to lists
        df1.append(diff_df1)
        customer_ids.append(customer_id)
    # Concatenate
    df1 = np.concatenate(df1, axis = 0)
    # Transform to dataframe
    df1 = pd.DataFrame(df1, columns = [col + '_diff1' for col in df[num_features].columns])
    # Add customer id
    df1['customer_ID'] = customer_ids
    return df1

In [20]:
train_diff = get_difference(df, num_features)

100%|██████████| 4575/4575 [00:15<00:00, 299.81it/s]


In [21]:
## categorical feature aggregation
train_cat_agg = df.groupby("customer_ID")[cat_vars].agg(['count', 'last', 'nunique']) # give summary statistics for each categrocial feature
train_cat_agg.columns = ['_'.join(x) for x in train_cat_agg.columns] # join the column name tuples to a single name
train_cat_agg.reset_index(inplace = True) # get the customer_ID in as a column again and reset index

In [22]:
df_agg = train_num_agg.merge(train_cat_agg, how = 'inner', on = 'customer_ID').merge(train_diff, how = 'inner', on = 'customer_ID').merge(target, how = 'inner', on = 'customer_ID')


In [23]:
df_agg

,customer_ID,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_mean,D_39_std,D_39_min,D_39_max,...,c_BBBB1_diff1,c_BBBB2_diff1,c_RRR0_diff1,c_RRR1_diff1,c_PD_348_diff1,c_PD_355_diff1,c_PD_439_diff1,c_PB_49_diff1,c_PR_41_diff1,target
0,000919ba92d9a04c28e1e49f6cd855ca36e1df7c79cc05...,0.499402,0.015600,0.476616,0.522344,0.498361,0.470466,0.198861,0.004601,0.830081,...,-0.008202,-0.315132,-2.043217,0.000802,-0.043284,-0.053059,-55.872944,-215.132233,-1.278214,0
1,00158cf08fcf7ec058529dd71b4cff04ce89314e79840b...,0.691427,0.024675,0.666900,0.733644,0.707521,0.026813,0.080476,0.000230,0.294493,...,-0.001197,-0.001025,1.261794,NaN,0.105418,0.002994,-1.430107,-3.275841,-1.123914,0
2,0018753794d55ff4ab24aaa7f8e65d504fd1efbf04e369...,0.281522,0.090988,0.198344,0.498234,0.325086,0.095091,0.185570,0.001569,0.567713,...,-0.855067,0.000209,-4.628440,-0.002885,0.123645,0.011405,-0.592819,-110.659798,-5.362189,1
3,0044d8693a5c204d5e22297b11e566d9de1c1610899d20...,0.863033,0.035847,0.808965,0.915033,0.878732,0.005821,0.002665,0.001335,0.009564,...,0.008055,0.036351,1.186878,-0.001879,-0.075793,-0.040460,-0.219735,1.817076,2.402584,0
4,00597bc3d552264d841bd1a52cfaf3ebe40755f96d85a5...,0.343063,0.047509,0.220225,0.378734,0.371411,0.164823,0.201824,0.009121,0.803373,...,NaN,0.042807,0.118287,-0.080717,0.058666,0.057578,-76.647720,-194.107788,-83.419312,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4570,ffbdeb593c97bc39bf6228e7236796cf5dcd530ef73014...,0.800766,0.054486,0.665637,0.855915,0.823051,0.551746,0.366059,0.004717,1.215695,...,1.759626,0.187208,-0.001998,0.005859,0.063895,0.040283,71.111038,0.162405,0.518457,0
4571,ffbff903bdd4104397101a428a4c5c4daa4d28e26f8716...,0.985665,0.020549,0.947392,1.004385,1.004385,0.121927,0.116296,0.000431,0.383494,...,-0.003195,-0.013913,0.512358,0.000810,-0.218525,-0.020217,-5.684210,-17.958992,-0.272193,0
4572,ffd4ac1fe3746d586e00906a56ae8a26fdad358f5b4789...,0.583752,0.079215,0.441809,0.716586,0.537270,0.303983,0.264887,0.001018,0.886130,...,0.008777,0.007561,5.159636,0.001569,0.011175,0.085185,41.172752,0.528886,-0.146537,0
4573,ffd6a301b0f94dec47dda9fec844cdc640a04bb38e8707...,0.371422,0.028512,0.332961,0.422435,0.375672,0.057875,0.083183,0.005337,0.243254,...,-1.823282,-1.243660,26.029510,NaN,NaN,NaN,-1.033933,-62.184582,-0.529305,1


In [24]:

del train_num_agg, train_cat_agg, train_diff
import gc
gc.collect()

0

In [25]:
cat_features = [f"{cf}_last" for cf in cat_vars]

In [26]:
encoder = OrdinalEncoder()

In [27]:
df_agg[cat_features] = encoder.fit_transform(df_agg[cat_features])

In [28]:
num_cols_mean = [col for col in df_agg.columns if 'mean' in col]
num_cols_last = [col for col in df_agg.columns if 'last' in col and col not in cat_features]


for col in range(len(num_cols_last)):
    try:
        df_agg[f'{num_cols_last[col]}_mean_diff'] = df_agg[num_cols_last[col]] - df_agg[num_cols_mean[col]]
    except:
        pass

In [29]:
df_agg

,customer_ID,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_mean,D_39_std,D_39_min,D_39_max,...,c_BBBB_last_mean_diff,c_BBBB1_last_mean_diff,c_BBBB2_last_mean_diff,c_RRR0_last_mean_diff,c_RRR1_last_mean_diff,c_PD_348_last_mean_diff,c_PD_355_last_mean_diff,c_PD_439_last_mean_diff,c_PB_49_last_mean_diff,c_PR_41_last_mean_diff
0,000919ba92d9a04c28e1e49f6cd855ca36e1df7c79cc05...,0.499402,0.015600,0.476616,0.522344,0.498361,0.470466,0.198861,0.004601,0.830081,...,-0.019302,0.083203,-0.179792,-0.031441,0.002298,-0.051647,-0.046471,-32.842237,-100.223597,-9.798897
1,00158cf08fcf7ec058529dd71b4cff04ce89314e79840b...,0.691427,0.024675,0.666900,0.733644,0.707521,0.026813,0.080476,0.000230,0.294493,...,-0.868411,-0.077614,-0.011059,-10.544500,0.039070,-0.004932,0.028528,0.998090,2.139511,2.418938
2,0018753794d55ff4ab24aaa7f8e65d504fd1efbf04e369...,0.281522,0.090988,0.198344,0.498234,0.325086,0.095091,0.185570,0.001569,0.567713,...,-0.508805,-1.227995,-0.604594,15.042042,-0.001188,-0.531127,-0.288131,-64.127581,-285.092994,-24.039815
3,0044d8693a5c204d5e22297b11e566d9de1c1610899d20...,0.863033,0.035847,0.808965,0.915033,0.878732,0.005821,0.002665,0.001335,0.009564,...,0.018584,0.010627,0.007818,10.131473,-0.013267,-0.008269,0.124844,0.455170,1.319511,1.569245
4,00597bc3d552264d841bd1a52cfaf3ebe40755f96d85a5...,0.343063,0.047509,0.220225,0.378734,0.371411,0.164823,0.201824,0.009121,0.803373,...,-0.024287,0.002214,-0.040757,-0.417466,-0.074078,0.166081,0.198613,-21.583628,-175.827881,-73.480048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4570,ffbdeb593c97bc39bf6228e7236796cf5dcd530ef73014...,0.800766,0.054486,0.665637,0.855915,0.823051,0.551746,0.366059,0.004717,1.215695,...,-0.016377,0.968971,0.156534,-5.786874,0.005488,-0.169338,-0.105161,-137.114383,-1.085928,-1.830852
4571,ffbff903bdd4104397101a428a4c5c4daa4d28e26f8716...,0.985665,0.020549,0.947392,1.004385,1.004385,0.121927,0.116296,0.000431,0.383494,...,-3.032281,-0.027999,0.001957,0.016325,0.003114,-0.080864,-0.000624,-86.080570,-65.432011,-4.938158
4572,ffd4ac1fe3746d586e00906a56ae8a26fdad358f5b4789...,0.583752,0.079215,0.441809,0.716586,0.537270,0.303983,0.264887,0.001018,0.886130,...,-0.174720,-0.055725,-0.003719,3.505672,-0.132084,0.729498,-0.100039,27.630789,-3.091750,-0.734702
4573,ffd6a301b0f94dec47dda9fec844cdc640a04bb38e8707...,0.371422,0.028512,0.332961,0.422435,0.375672,0.057875,0.083183,0.005337,0.243254,...,2.182336,-1.354337,-0.807737,19.895425,-0.000386,0.416058,0.365047,-0.133332,-4.992001,-0.076932


In [43]:
def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    ## TWEAK
    y_pred = pd.Series(y_pred, index=y_true.index)

    y_true = pd.DataFrame(y_true)
    y_pred = pd.DataFrame(y_pred)

    y_true = y_true.rename(columns={y_true.columns[0]:'target'})
    y_pred = y_pred.rename(columns={y_pred.columns[0]:'prediction'})
    ##

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

def build_param_dict(boosting_type='gbdt', # params can be found here https://lightgbm.readthedocs.io/en/v3.3.2/Parameters.html
                     max_depth=-1,
                     n_estimators=100,
                     num_leaves=31,
                     class_weight = None, # might want to try is_unbalance
                     learning_rate= 0.1,
                     min_data_in_leaf=20,
                     bagging_fraction=1.0,
                     feature_fraction=1.0,
                     objective='binary',
                     reg_alpha=0.,
                     reg_lambda=0.,
                     categorical_features=cat_features,
                     random_state=None, 
                     **kwargs
                     ):

    return dict(
        boosting_type=boosting_type, # params can be found here https://lightgbm.readthedocs.io/en/v3.3.2/Parameters.html
        max_depth=max_depth,
        n_estimators=n_estimators,
        num_leaves=num_leaves,
        class_weight = class_weight, # might want to try is_unbalance
        learning_rate= learning_rate,
        min_data_in_leaf=min_data_in_leaf,
        bagging_fraction=bagging_fraction,
        feature_fraction=feature_fraction,
        objective=objective,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        categorical_feature=categorical_features,
        random_state=random_state,
        **kwargs
        )


def initialize_model(param_dict=build_param_dict()):

    return lgb.LGBMClassifier(**param_dict)

def train_model(model,
                X: np.ndarray,
                y: np.ndarray,
                eval_metric=make_scorer(amex_metric),
                early_stopping=30,
                num_splits=1,
                test_size = 0.3,
                init_model = None, #(str, pathlib.Path, Booster, LGBMModel or None, optional (default=None)) – Filename of LightGBM model, Booster instance or LGBMModel instance used for continue training.
                ):

    ## make sure the passed data is aggregated, otherwise it is going to split row/wise, not ustomer wise

    # check out init score and init model


    eval_sets = []

    for i in range(num_splits):

        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=test_size)
        eval_sets.append((X_val, y_val))

    model.fit(X, y, early_stopping_rounds=early_stopping, eval_metric=eval_metric, eval_set=eval_sets, init_model=None)

    return model

# Experimenting with models

In [31]:
# ====================================================
# Configurations
# ====================================================
class CFG:
    seed = 42
    n_folds = 5
    target = 'target'
    boosting_type = 'dart'
    metric = 'binary_logloss'

# ====================================================
# Seed everything
# ====================================================
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

In [32]:
X = df_agg.drop(columns=['customer_ID', 'target'])
y = df_agg['target']
X_train, X_val, y_train1, y_val1 = train_test_split(X, y)

In [33]:
params = {
    'objective': 'binary',
    'metric': CFG.metric,
    'boosting': CFG.boosting_type,
    'seed': CFG.seed,
    'num_leaves': 100,
    'learning_rate': 0.01,
    'feature_fraction': 0.20,
    'bagging_freq': 10,
    'bagging_fraction': 0.50,
    'n_jobs': -1,
    'lambda_l2': 2,
    'min_data_in_leaf': 40,
    }

In [39]:
import pickle

In [42]:
features = X_train.columns

In [41]:
# ====================================================
# Amex metric
# ====================================================
def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)

# ====================================================
# LGBM amex metric
# ====================================================
def lgb_amex_metric(y_pred, y_true):
    y_true = y_true.get_label()
    return 'amex_metric', amex_metric(y_true, y_pred), True

In [45]:
# Create a numpy array to store test predictions
test_predictions = np.zeros(len(X_train))
# Create a numpy array to store out of folds predictions
oof_predictions = np.zeros(len(X_train))
kfold = StratifiedKFold(n_splits = CFG.n_folds, shuffle = True, random_state = CFG.seed)

features = X_train.columns


In [ ]:
lgb.Dataset()

In [52]:
for fold, (trn_ind, val_ind) in enumerate(kfold.split(X_train, y_train1)):
    print(val_ind)

[  16   18   24   30   33   36   48   50   53   54   58   66   68   74
   77   83   85   89   92   95   96  101  102  103  108  113  116  122
  131  136  171  174  177  184  185  191  195  201  206  209  212  216
  218  234  237  254  258  259  263  272  274  281  286  290  294  295
  299  305  306  311  320  336  338  340  346  353  355  357  359  361
  368  369  371  372  376  379  383  394  399  402  412  415  421  422
  423  431  438  441  445  450  465  466  472  477  478  479  481  482
  487  495  501  503  517  519  526  530  537  539  545  552  553  554
  555  556  560  565  572  573  577  581  585  589  592  595  596  601
  611  615  619  620  624  628  629  633  634  636  649  650  656  662
  663  664  665  671  675  678  679  683  690  693  697  704  714  715
  721  747  749  753  757  761  764  766  772  773  780  782  784  788
  789  790  792  794  800  806  825  831  836  838  849  850  857  858
  861  863  864  870  871  877  885  889  891  892  896  898  909  917
  918 

In [ ]:
for fold, (trn_ind, val_ind) in enumerate(kfold.split(X_train, y_train1)):
    print(' ')
    print('-'*50)
    print(f'Training fold {fold} with {len(features)} features...')
    x_train, x_val = X_train[features].iloc[trn_ind], X_train[features].iloc[val_ind]
    y_train, y_val = y_train1.iloc[trn_ind], y_train1.iloc[val_ind]
    lgb_train = lgb.Dataset(x_train, y_train, categorical_feature = cat_features)
    lgb_valid = lgb.Dataset(x_val, y_val, categorical_feature = cat_features)
    model = lgb.train(
        params = params,
        train_set = lgb_train,
        num_boost_round = 10500,
        valid_sets = [lgb_train, lgb_valid],
        early_stopping_rounds = 1500,
        verbose_eval = 500,
        feval = lgb_amex_metric
        )
    # Save best model
    pickle.dump(model, f'/home/slawa/code/code-rep0/projects/AMEX_default_prediction/pickles/lgbm_{CFG.boosting_type}_fold{fold}_seed{CFG.seed}.pkl')
        # Predict validation
    val_pred = model.predict(x_val)
    # Add to out of folds array
    oof_predictions[val_ind] = val_pred
    # Predict the test set
    test_pred = model.predict(test[features])
    test_predictions += test_pred / CFG.n_folds # that is how it gets more or less a probability
    # Compute fold metric
    score = amex_metric(y_val, val_pred)
    print(f'Our fold {fold} CV score is {score}')
    del x_train, x_val, y_train, y_val, lgb_train, lgb_valid
    gc.collect()

In [37]:
model_lgb = initialize_model(build_param_dict(categorical_features=None))

In [38]:
model_lgb.fit(X_train, y_train1)

LGBMClassifier(bagging_fraction=1.0, categorical_feature=None,
               feature_fraction=1.0, min_data_in_leaf=20, objective='binary')

In [39]:
y_pred = model_lgb.predict_proba(X_val)[:, 1]

In [44]:
amex_metric(y_val1, y_pred)

0.67306840893694

In [130]:
# cat_imputer = SimpleImputer(strategy="most_frequent") ## replace with KNNimputer on one neighbour, after transforming to numericals
# #cat_imputer = KNNImputer(n_neighbors=1) # introducing it did not improve performance, but is computationally demanding
# cat_encoder = OrdinalEncoder()
# #cat_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore') ## what happens to the old columns?
# cat_pipe = make_pipeline(cat_imputer, cat_encoder)

In [131]:
# preprocessor= ColumnTransformer([
#         ('cat_pip', cat_pipe, cat_features)],
#         remainder='passthrough' ## all columns not in num_vars and red_cat_vars are dropped.
#             )

In [132]:
# X_pp = preprocessor.fit_transform(X)

In [79]:
model_lgb.fit(X_train, y_train)

[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0


LGBMClassifier(bagging_fraction=1.0, categorical_feature=None,
               feature_fraction=1.0, min_data_in_leaf=20, objective='binary')

In [81]:
y_pred = model_lgb.predict_proba(X_val)[:, 1]

In [82]:
amex_metric(y_val, y_pred)

0.7123541944359542

In [76]:
amex_metric(y_val, y_pred)

0.7143069147981995

In [ ]:
# params = {
#         'objective': 'binary',
#         'metric': CFG.metric,
#         'boosting': CFG.boosting_type,
#         'seed': CFG.seed,
#         'num_leaves': 100,
#         'learning_rate': 0.01,
#         'feature_fraction': 0.20,
#         'bagging_freq': 10,
#         'bagging_fraction': 0.50,
#         'n_jobs': -1,
#         'lambda_l2': 2,
#         'min_data_in_leaf': 40,
#         }